In [1]:
!pip install langchain langchain-community sentence-transformers langchain-text-splitters pypdf chromadb langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66

In [2]:
!pip install -q langchain-text-splitters

In [3]:
import os
import getpass
os.environ["API_KEY"] = getpass.getpass('Enter key: ')

Enter key: ··········


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

embedding_model = HuggingFaceEmbeddings(
    model_name = "all-MiniLM-L6-v2"
)
chat_model = ChatOpenAI(
    model= "llama-3.1-8b-instant",
    api_key = os.environ["API_KEY"],
    base_url = "https://api.groq.com/openai/v1"
)

/tmp/ipython-input-2659725199.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
loader = PyPDFLoader("AU-Student-Handbook-2025.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 200
)

chunks = text_splitter.split_documents(documents)



In [6]:
vector_db = Chroma.from_documents(
        chunks,
        embedding=embedding_model)




In [9]:
query = "What are the rules and regulations of this university?"

retrived_docs = vector_db.similarity_search(query=query,k=5)

context = "\n".join(docs.page_content for docs in retrived_docs)

final_prompt = f"""
Behave as an professional university chatbot agent.
Only answer the query using the given content only.
If you don't know any answer then just say i don't know.
Don't say to go to any university website.
Also mention the particular used context line below every explained point.
Do not add introductory sentences.
Start directly with bullet points from context.
Do not paraphrase. Use exact wording from the context.


context:
{context}

query:
{query}
"""

In [10]:
print(f"Query: {query} Answer : \n {chat_model.invoke(final_prompt).content}")

Query: What are the rules and regulations of this university? Answer : 
 * The sale, possession, distribution, manufacture, and use of alcohol, tobacco, and/or other controlled substances is expressly prohibited on University premises. (The sale, possession, distribution, manufacture, and use of alcohol, tobacco, and/or other controlled substances is expressly prohibited on University premises.)
* Students found in violation of this directive shall be subject to disciplinary measures within the University, including but not limited to expulsion from campus housing and forfeiture of Financial Aid, as well as arrest and criminal prosecution in accordance with local, state and/or national law. (Students found in violation of this directive shall be subject to disciplinary measures within the University, including but not limited to expulsion from campus housing and forfeiture of Financial Aid, as well as arrest and criminal prosecution in accordance with local, state and/or national law.)